# Домашнее задание по рекомендательным системам

В данном домашнем задании вам предлагается реализовать User-based рекомендательную систему. Так же требуется реализовать несколько вспомогательных функций, шаблоны которых вы можете найти в `utils.py`.

Требования к выполнению задания:
- Реализация функции из `utils.py` засчитывается, только если пройдены все соответствующие тесты из `test.py`. Запуск тестов: <font color='red'>pytest test.py</font>. Для тестов вам потребуются библиотеки `numpy`, `scipy`, `pytest` и `hypothesis`.
- Плагиат запрещен. Если будет замечено, что часть задания списана, то 0 баллов ставится как списывающему, так и давшему списать.
- Если пользуетесь кодом из открытых источников, то указывайте ссылки, откуда взяли решение. Иначе такой код может быть воспринят как плагиат.
- При выполнении задания нельзя использовать библиотеку `scipy` и функцию `numpy.linalg.norm`

При запуске тестов могут появиться предупреждения: PearsonRConstantInputWarning и PearsonRNearConstantInputWarning. На них можно не обращать внимания.

Возможный максимум баллов за задание: 10 баллов <br>
Дедлайн: ??? <br>
Штраф: ??? - будет ли в курсе штраф? <br>
<br>
Для ускорения проверки, напишите здесь получившееся количество баллов: ...

In [3]:
from utils import euclidean_distance, euclidean_similarity, pearson_similarity, apk, mapk
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

## 1. Метрика сходства
<b>1.1. Реализация метрик (2 балла)</b>

Первое, с чем необходимо разобраться, при реализации User-based подхода, это с метрикой, с помощью которой будет решаться, насколько похожи пользователи. Вам предлагается реализовать 2 метрики: на основе евклидовой метрики и коэффициент корреляции Пирсона. Шаблоны для обоих функций можете найти в `utils.py`. Не забудьте проверить реализацию на тестах.

Евклидова метрика:
\begin{equation}
d(p,q)=\sqrt{(p_1-q_1)^2+(p_2-q_2)^2+\dots+(p_n-q_n)^2} = \sqrt{\sum_{k=1}^n (p_k-q_k)^2}
\end{equation}

В этом случае $d(p, q) \in [0, \infty)$, при этом если $d(p, q) \to 0$, то $sim(p, q) \to 1$. С учетом этого конечная формула будет выглядеть следующим образом:
\begin{equation}
sim(p, q) = \frac{1}{1 + d(p, q)}
\end{equation}
Так же в этой формуле не будет проблем с делением на 0.

Коэффициент корреляции Пирсона:
\begin{equation}
r_{xy} = \frac {\sum_{i=1}^{m} \left( x_i-\bar{x} \right)\left( y_i-\bar{y} \right)}{\sqrt{\sum_{i=1}^{m} \left( x_i-\bar{x} \right)^2 \sum_{i=1}^{m} \left( y_i-\bar{y} \right)^2}}
\end{equation}

In [4]:
!pytest test.py

============================= test session starts =============================
platform win32 -- Python 3.8.8, pytest-6.2.4, py-1.10.0, pluggy-0.13.1
rootdir: d:\Google\Projects\ml_tinkoff_2021\notebooks\lesson_10_recommend\hw
plugins: anyio-2.2.0, hypothesis-6.34.2
collected 11 items

test.py F..........                                                      [100%]

================================== FAILURES ===================================
___________________________ test_euclidean_distance ___________________________

    @given(same_len_lists())
>   def test_euclidean_distance(lists):

test.py:29: 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _

self = <hypothesis.core.StateForActualGivenExecution object at 0x000001AD1BC3A310>
message = 'Hypothesis test_euclidean_distance(lists=[[0.0], [0.0]]) produces unreliable results: Falsified on the first call but did not on a subsequent one'

    def __flaky(self, message):
        if len(self.falsifying_e

<b>1.2. (1 балл)</b>

Рассмотрим пользователей $u$ и $v$. Им соотвествуют векторы $x_u$ и $x_v$, где $x_u[i] = r_{ui}$ и $x_v[i] = r_{vi}$. Из лекции известно, что похожесть между векторами $x_u$ и $x_v$ вычисляются только для тех индексов i, для которых существует и $r_{ui}$, и $r_{vi}$. То есть верно следуюющее:
\begin{equation}
sim(u, v) = sim(x_uI_{uv}, x_vI_{uv}),
\end{equation}
где $I_{uv} = [i | \exists r_{ui} \& \exists r_{vi}]$. При этом если $I_{uv} = \emptyset$, то $sim(u, v) \to -\infty$.

Реализуйте два новых метода, которые переиспользуют написанные вами `euclidean_distance` и `pearson_distance`, добавляющие условия на $x_u$ и $x_v$. Считается, что $x_u[i] = 0$, если $\nexists r_{ui}$. То же верно для $x_v$.

При реализации заданий можно как написать новые функции, так и использовать декораторы.

In [5]:
def intersection_items(f):
    def decorator(x: np.array, y: np.array):
        indices = np.arange(len(x))
        intresec_idx = indices[(x != 0) & (y != 0)]
        return f(x[intresec_idx], y[intresec_idx])
    return decorator

In [6]:
# your code (ﾉ>ω<)ﾉ :｡･:*:･ﾟ’★,｡･:*:･ﾟ’☆
@intersection_items
def euclidean_similarity(x: np.array, y: np.array) -> float:
    """
    Calculate euclidean similarity between points x and y
    Args:
        x, y: two points in Euclidean n-space
    Returns:
        Similarity between points x and y
    """
    return 1 / (1 + euclidean_distance(x, y))

@intersection_items
def pearson_similarity(x: np.array, y: np.array) -> float:
    """
    Calculate a Pearson correlation coefficient given 1-D data arrays x and y
    Args:
        x, y: two points in n-space
    Returns:
        Pearson correlation between x and y
    """
    f = lambda z: np.sum((z - z.mean()) ** 2)

    return np.sum((x - x.mean()) * (y - y.mean())) / np.sqrt(f(x) * f(y))

## 2. User-based method
<b>2.1. (3 балла)</b> 

Реализовать User-based подход, реализовав методы класса `UserBasedRecommendation`, основанного на использовании `NearestNeighbors`. В качестве метрики может для нахождения похожих пользователей может быть использована как евклидова метрика, так и коэффициент корреляции Пирсона.

Не забывайте, что `NearestNeighbors` ищет минимум расстояния между элементами, поэтому логично в качестве метрики при инициализации `NearestNeighbors` использовать обратную метрике схожести. То есть такую, что когда $sim(u, v) \to 1$, то $d(u, v) \to 0$. Например: $d(u, v) = 1 - sim(u, v)$

In [7]:
from sklearn.neighbors import NearestNeighbors
from typing import Optional

from collections import Counter


class UserBasedRecommendation:
    def __init__(self, metric: str = 'euclidean', n_recommendations: int = 5, alpha: float = 0.8):
        """
        Args:
            metric: name of metric: ['euclidean', 'pearson']
            n_recommendations: number of recommendations. Also can be specified self.make_recommendation
            alpha: similarity threshold: if sim(u, v) > alpha then u and v are similar
        """
        self.metric = metric
        self.n_recommend = n_recommendations
        self.threshold = alpha

    def fit(self, X: np.array):
        """
        Args:
            X: matrix N x M where X[u, i] = r_{ui} if r_{ui} exists else X[u, i] = 0
        """
        self.X = X
        self.neighbors = NearestNeighbors(metric=self.distance, radius=self.threshold)
        self.neighbors.fit(self.X)

    def __find_closest_users(self, user_id: int, n_closest_users: int):
        closer_users = self.neighbors.radius_neighbors(
            X=[self.X[user_id]], 
            sort_results=True
        )

        return closer_users[1][:n_closest_users][0]

    def make_recommendation(self, user_id: int, n_recommendations: Optional[int] = None, n_neighbors: int = 2, verbose=True):
        """
        Args:
            user_id: user id to whom you want to recommend
            n_recommendations: number of recommendations
        """
        if n_recommendations is not None:
            self.n_recommend = n_recommendations

        users_closer = self.__find_closest_users(user_id, n_neighbors)

        if verbose:
            print(f"{n_neighbors} neighbours of {user_id}: {users_closer}")

        # создаем маску, означающая 1 если товар куплен и 0 в противном случае
        mask_items = self.X[users_closer, :] != 0

        # преобразуем в формат чисел и суммируем (то есть для каждого товара количество покупок)
        num_items = mask_items.astype(int) if n_neighbors == 1 else mask_items.astype(int).sum(0)

        # сортируем, тем самым находим самые популярные товары у данного типа пользователей
        return np.argsort(num_items / len(users_closer)).squeeze()[::-1][:self.n_recommend]

    def distance(self, x: np.array, y: np.array):
        if self.metric == 'euclidean':
            return 1 - euclidean_similarity(x, y)
        elif self.metric == 'pearson':
            return 1 - pearson_similarity(x, y)
        else:
            raise ValueError("You can only use metrics 'euclidean' or 'pearson'")

Хотим сделать рекомендации для фиксированного пользователя $u_{0}$ Найдем множество $U(u_{0})$ пользователей, похожих на данного:
\begin{equation}
U(u_{0}) = {u \in U|sim(u_{0}, u) > \alpha}
\end{equation}
За это отвечает `NearestNeighbors`. $\alpha$ я не меняла, так как если $sim(u, v) > \alpha$, то $distance < alpha$, таким образом radius = alpha

После этого для каждого товара вычислим, как часто пользователи из множества $U(u_{0})$ покупали его:
\begin{equation}
p_{i} = \frac{|\{u \in U(u_{0})|\exist r_{ui}\}|}{|U(u_{0}|}
\end{equation}

Далее сортируем в порядке убывания и выбираем заданное количество рекомендаций.

**Источник**
Из конспекта

<b>2.2. (1 балла)</b>

Приведите пример, для которого использование разных метрик будет давать разные рекомендации. Объясните свой пример.

In [8]:
# your code (ﾉ>ω<)ﾉ :｡･:*:･ﾟ’★,｡･:*:･ﾟ’☆
X = np.array([
    [3, 4, 8, 9],
    [3, 5, 0, 3], # 1 user_id
    [8, 7, 6, 2], # 2
    [8, 6, 1, 5], # 3
])

In [9]:
recommend = UserBasedRecommendation(metric='euclidean', n_recommendations=2, alpha=0.85)
recommend.fit(X)
recommend.make_recommendation(user_id=3)

2 neighbours of 3: [3 1]


array([3, 1], dtype=int64)

In [10]:
recommend = UserBasedRecommendation(metric='pearson', n_recommendations=2, alpha=0.85)
recommend.fit(X)
recommend.make_recommendation(user_id=3)

2 neighbours of 3: [3 2]


array([3, 2], dtype=int64)

**Объяснение:** рекомендации пользователю с user_id 3 (пронумеровали пользователей и товары от 0) у евклидовой метрики и коэффициент корреляции Пирсона различный, так как и соседи тоже различны. У каждого есть пользователь 3 (то есть пользователь больше всего похож на самого себя). Расмотрим каждый алгоритм по отдельности:
- у евклидовой метрики в рекомендациях следует пользователь user_id 1. Если посмотреть на товары, то первый user дал 0-ому товару оценку ниже, чем 1-ому, а у третьего пользователя ситуация обстоит иначе. Более того, третий пользователь купил 2 товар, а первый отказался.
- коэффициент корреляции Пирсона учитывает эти различия, поэтому для третьего пользователя соседом является 2 пользователь.

**Выводы**
Евклидова метрика не учитвает различия между оценками для каждого товара, у него учитывается просто общая сумма различий. Коэффициент корреляции Пирсона более чувствителен к таким особенностям.

## 3. Оценка качества
<b>3.1. (1 балл)</b>

Реализуйте Average Precision at k и Mean Average Precision at k. Шаблоны можете найти в `utils.py`.
\begin{align*}
AP@K = \frac{1}{m}\sum_{k=1}^K P(k)*rel(k), \\
MAP@K = \frac{1}{|U|}\sum_{u=1}^{|U|}(AP@K)_u
\end{align*}
где $P(k)$ - Precision at k, $rel(k) = 1$, если рекомендация релевантна, иначе $rel(k) = 0$.

In [11]:
!pytest test.py

============================= test session starts =============================
platform win32 -- Python 3.8.8, pytest-6.2.4, py-1.10.0, pluggy-0.13.1
rootdir: d:\Google\Projects\ml_tinkoff_2021\notebooks\lesson_10_recommend\hw
plugins: anyio-2.2.0, hypothesis-6.34.2
collected 11 items

test.py ...........                                                      [100%]

============================== warnings summary ===============================
..\..\..\..\..\..\Programs\Anaconda\lib\site-packages\pyreadline\py3k_compat.py:8
  D:\Programs\Anaconda\lib\site-packages\pyreadline\py3k_compat.py:8: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
    return isinstance(x, collections.Callable)

test.py::test_pearson_similarity
  d:\Google\Projects\ml_tinkoff_2021\notebooks\lesson_10_recommend\hw\utils.py:38: RuntimeWarning: invalid value encountered in double_scalars
    return

## 4. Применение модели
<b>4.1. (2 балла)</b>

Выгрузите датасет `ratings_small.csv`: https://www.kaggle.com/rounakbanik/the-movies-dataset#ratings_small.csv

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [13]:
data = pd.read_csv('ratings_small.csv', index_col=False)
data.shape

(100004, 4)

In [14]:
data.userId.min(), data.userId.max(), len(data.userId.unique())

(1, 671, 671)

In [15]:
data.movieId.min(), data.movieId.max(), len(data.movieId.unique())

(1, 163949, 9066)

Для простоты работы с данными, измените нумерацию пользователей и фильмов так, чтобы нумерация начиналась с 0 и шла непрерывно.

In [16]:
# your code (ﾉ>ω<)ﾉ :｡･:*:･ﾟ’★,｡･:*:･ﾟ’☆
user_to_idx = {user_id : idx for idx, user_id in enumerate(data.userId.unique())}
movie_to_idx = {movie_id : idx for idx, movie_id in enumerate(data.movieId.unique())}

In [17]:
replacing_user = lambda x: user_to_idx[x]
replacing_movie = lambda x: movie_to_idx[x]

In [18]:
data.userId = data.userId.apply(replacing_user)
data.movieId = data.movieId.apply(replacing_movie)

In [19]:
data.userId.min(), data.userId.max(), len(data.userId.unique())

(0, 670, 671)

In [20]:
data.movieId.min(), data.movieId.max(), len(data.movieId.unique())

(0, 9065, 9066)

Удалим для наиболее активных пользователей 5 оценок

In [21]:
active_users = data.userId.value_counts()[:10].index
test_data = pd.DataFrame([], columns=data.columns)
for user_id in active_users:
    _, test = train_test_split(data[data.userId == user_id], test_size=5, random_state=42)
    test_data = test_data.append(test, ignore_index=True)
    data = data[~((data.userId == user_id) & (data.movieId.isin(test.movieId.values)))]
data.shape, test_data.shape

((99954, 4), (50, 4))

Преобразуем данные в таблицу `X`, с которой может работать `UserBasedRecommendation`, где $X_{ui} = r_{ui}$, если пользователь $u$ поставил оценку фильму $i$, и $X_{ui} = 0$, если пользователь $u$ не проставил оценку фильму $i$.

Вам может пригодиться `csr_matrix`.

In [22]:
# your code (ﾉ>ω<)ﾉ :｡･:*:･ﾟ’★,｡･:*:･ﾟ’☆
from scipy.sparse import csr_matrix, coo_matrix

In [23]:
users = data['userId'].values
movies = data['movieId'].values
rating = data['rating'].values

In [24]:
X = csr_matrix((rating, (users, movies)), shape=(users.max()+1, movies.max()+1)).todense().A

In [25]:
X

array([[2.5, 3. , 3. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       ...,
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ]])

Для пользователей, у которых были удалены фильмы, найдите топ 100 фильмов, который должен посмотреть каждый из этих пользователей, используя `UserBasedRecommendation`. Не забудьте подобрать параметр alpha.

In [26]:
actual = []
for user in active_users:
    actual.append(test_data[test_data.userId == user].movieId.values)

In [27]:
active_users

Int64Index([546, 563, 623, 14, 72, 451, 467, 379, 310, 29], dtype='int64')

In [28]:
# your code (ﾉ>ω<)ﾉ :｡･:*:･ﾟ’★,｡･:*:･ﾟ’☆
def recommendations(metric: str, alpha: int = 0.8, n_neighbors: int = 10):
    user_recommend = UserBasedRecommendation(metric=metric, n_recommendations=100, alpha=alpha)
    user_recommend.fit(X)

    predicted = []
    for user in active_users:
        predicted.append(user_recommend.make_recommendation(user_id=user, n_neighbors=n_neighbors, verbose=False))
    return predicted

Используя метрику `MAP@5`, `MAP@10` и `MAP@100`, определите, насколько эффективна user-based рекомендательная система для данной задачи.

In [29]:
from utils import mapk

In [30]:
def all_mapk(actual, predicted):
    print(f"MAP@5: {mapk(actual, predicted, 5)}")
    print(f"MAP@10: {mapk(actual, predicted, 10)}")
    print(f"MAP@100: {mapk(actual, predicted, 100)}")

In [ ]:
print("Euclidean")
all_mapk(actual, recommendations('euclidean', alpha=0.8))

Euclidean
MAP@5: 0.01
MAP@10: 0.005
MAP@100: 0.0005


In [ ]:
print("Pearson")
all_mapk(actual, recommendations('pearson', alpha=0.7))

Pearson
MAP@5: 0.02
MAP@10: 0.01142857142857143
MAP@100: 0.0012211553473848557


Грустно, результат очень плохой. Причем с увеличением k, map@k уменьшается.

Как можно улучшить работу модели?

<b>Ответ:</b> Подобрать гиперпараметры. Например, alpha или количество соседей.

In [31]:
alphas = np.arange(0, 1, 0.1)
n_neighbors = np.arange(5, 100, 5)

In [39]:
print("Euclidean")
best_alpha = 0
best_n = 0
best_result = 0
for alpha in alphas:
    for n in n_neighbors:
        predicted = recommendations('euclidean', alpha=alpha, n_neighbors=n)
        result = mapk(actual, predicted, 5)
        if result > best_result:
            best_result = result
            best_alpha = alpha
            best_n = n

Euclidean


In [40]:
print(f"Best alpha: {best_alpha}")
print(f"Best n_neighbors: {best_n}")
print(f"Best result: {best_result}")

Best alpha: 0.9
Best n_neighbors: 5
Best result: 0.024


In [32]:
print("Pearson")
best_alpha = 0
best_n = 0
best_result = 0
for alpha in alphas:
    for n in n_neighbors:
        predicted = recommendations('pearson', alpha=alpha, n_neighbors=n)
        result = mapk(actual, predicted, 5)
        if result > best_result:
            best_result = result
            best_alpha = alpha
            best_n = n

Pearson


In [33]:
print(f"Best alpha: {best_alpha}")
print(f"Best n_neighbors: {best_n}")
print(f"Best result: {best_result}")

Best alpha: 0.7000000000000001
Best n_neighbors: 5
Best result: 0.02


Так же я попыталась сама создать функцию нахождения соседей и другой отбор рекомендаций на основе рейтинга. То есть в предыдущем мы учитывали только купил\не купил, а тут сам рейтинг. ТО есть смотрим, какому товару пользователь поставил бы самый высокий рейтинг. 

**Источник** Вчера я написала, вкладку закрыла, но где-то на хабре, найти не могу. В целом, эта статья https://habr.com/ru/company/surfingbird/blog/139518/ тоже содержит данную формулу и может являться источником.

In [43]:
from sklearn.neighbors import NearestNeighbors
from typing import Optional

from collections import Counter


class UserBasedRecommendation:
    def __init__(self, metric: str = 'euclidean', n_recommendations: int = 5, alpha: float = 0.8):
        """
        Args:
            metric: name of metric: ['euclidean', 'pearson']
            n_recommendations: number of recommendations. Also can be specified self.make_recommendation
            alpha: similarity threshold: if sim(u, v) > alpha then u and v are similar
        """
        self.metric = metric
        self.n_recommend = n_recommendations
        self.threshold = alpha
        self.sim = self.define_sim()

    def fit(self, X: np.array):
        """
        Args:
            X: matrix N x M where X[u, i] = r_{ui} if r_{ui} exists else X[u, i] = 0
        """
        self.X = X

    def __find_closest_users(self, user_id: int, n_closest_users: int):
        closer_users = []
        for user in range(self.X.shape[0]):
            similarity = self.sim(self.X[user_id], self.X[user])
            if similarity > self.threshold:
                closer_users.append((similarity, user))

        sorted_closer = sorted(closer_users, reverse=True)[:n_closest_users]
        return [user for sim, user in sorted_closer]
        

    def make_recommendation(self, user_id: int, n_recommendations: Optional[int] = None, n_neighbors: int = 2, verbose=True):
        """
        Args:
            user_id: user id to whom you want to recommend
            n_recommendations: number of recommendations
        """
        if n_recommendations is not None:
            self.n_recommend = n_recommendations

        users_closer = self.__find_closest_users(user_id, n_neighbors)

        if verbose:
            print(f"{n_neighbors} neighbours of {user_id}: {users_closer}")

        ratings = []
        # среднее для пользователя (учитываем только купленные товары)
        mean_user_id = np.mean(self.X[user_id][self.X[user_id] != 0])
        
        for item in range(self.X.shape[1]):
            if self.X[user_id, item] == 0:
                temp = []
                sum_sim = 0
                for user in users_closer:
                    similarity = self.sim(self.X[user_id], self.X[user])
                    temp.append(similarity * (self.X[user, item] - np.mean(self.X[user])))
                    sum_sim += np.abs(similarity)
                ratings.append((mean_user_id + np.sum(temp) / sum_sim, item))

        sorted_ratings = sorted(ratings, reverse=True)[:self.n_recommend]
        return [item for rate, item in sorted_ratings]

    def define_sim(self):
        if self.metric == 'euclidean':
            return euclidean_similarity
        elif self.metric == 'pearson':
            return pearson_similarity
        else:
            raise ValueError("You can only use metrics 'euclidean' or 'pearson'")

    def distance(self, x: np.array, y: np.array):
        return 1 - self.sim(x, y)

In [44]:
print("Euclidean")
all_mapk(actual, recommendations('euclidean', alpha=0.3, n_neighbors=50))

Euclidean
MAP@5: 0.0
MAP@10: 0.0016666666666666666
MAP@100: 0.00025674763832658564


In [45]:
print("Pearson")
all_mapk(actual, recommendations('pearson', alpha=0.2, n_neighbors=50))

Pearson
MAP@5: 0.026666666666666665
MAP@10: 0.017499999999999998
MAP@100: 0.0019107015021134419
